In [1]:
from transformers import AutoTokenizer, BertModel
from transformers.utils import PaddingStrategy
from torch import nn
import torch

/home/pico/myCodes/medfusion/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("StanfordAIMI/RadBERT")
model = BertModel.from_pretrained("StanfordAIMI/RadBERT").to('cuda')

Some weights of the model checkpoint at StanfordAIMI/RadBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
class RadBertEmbedder(nn.Module):
    _device = f"{torch.device('cuda' if torch.cuda.is_available() else 'cpu')}:{torch.cuda.current_device()}"
    _tokenizer = tokenizer
    _model = model

    def __init__(self, emb_dim=32,*args, **kwargs):
        super().__init__()
        self.emb_dim = emb_dim
        self.mlp = nn.Sequential(
            nn.Linear(768, emb_dim), # 768 bert output的维度
            nn.LayerNorm(emb_dim),
            nn.ReLU(),
            nn.Linear(emb_dim, emb_dim),
            nn.LayerNorm(emb_dim),
        ).to(self._device)

    def forward(self, condition):
        inputs = self._tokenizer(condition, return_tensors="pt", padding=True)
        inputs = inputs.to(self._device)
        outputs = self._model(**inputs)
        c = outputs.last_hidden_state[:, 0] # [CLS]
        c = self.mlp(c)
        return c

In [12]:
emb_obj = RadBertEmbedder(emb_dim=1024)
condition = [
    "A photo of a lung xray with Edema,Pneumonia and Pleural Effusion",
    "A photo of a lung xray with Lung Opacity,Pneumonia,Pneumothorax and Pleural Effusion",
    "A photo of a lung xray with Fracture",
    "A photo of a lung xray with Cardiomegaly,Consolidation and Support Devices",
    "A photo of a lung xray with Atelectasis and Pneumothorax",
    "A photo of a lung xray with Support Devices",
    "A photo of a lung xray with Lung Opacity,Pleural Effusion and Support Devices",
    "A photo of a lung xray with Consolidation,Atelectasis and Pleural Effusion",
    "A photo of a lung xray with Pleural Other and Support Devices",
    "A photo of a lung xray with Lung Opacity,Edema and Pleural Effusion",
    "A photo of a lung xray with Edema,Atelectasis and Pleural Effusion",
    "A photo of a lung xray with Lung Opacity and Consolidation",
    "A photo of a lung xray with Cardiomegaly,Lung Opacity,Edema,Pleural Effusion and Support Devices",
    "A photo of a lung xray with Lung Opacity,Edema,Pleural Effusion and Support Devices",
    "A photo of a lung xray with Lung Opacity and Pleural Effusion",
    "A photo of a lung xray",
]
c = emb_obj(condition)
c.shape


tensor([  101,   170,  6307,  1104,   170, 13093,   193,  6447,  1114,  5048,
        14494,   117, 20673,  1105,   185,  1513, 12602,   174,  3101, 17268,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])
torch.Size([98])


torch.Size([16, 1024])